# Small script to compare years of inventories and extrapolate the emissions


The first part is used to compare different inventory years.
The second part is used to extrapolate the emissions from the most recent inventory year to the current year.

In [ ]:
from math import floor
from pathlib import Path
import copy

from emiproc.inventories.utils import group_categories, validate_group
from emiproc.grids import SwissGrid, LV95, WGS84
from emiproc.inventories.swiss import SwissRasters
from emiproc.inventories.zurich import MapLuftZurich
from emiproc.inventories.zurich.gnrf_groups import ZH_2_GNFR
from emiproc.regrid import remap_inventory, weights_remap, get_weights_mapping
from emiproc.utilities import SEC_PER_YR
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point
from emiproc.inventories.utils import add_inventories, scale_inventory
from emiproc.inventories.utils import get_total_emissions
from emiproc.speciation import speciate_inventory
from emiproc.regrid import remap_inventory
from emiproc.inventories.utils import crop_with_shape
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
path = Path(r"C:\Users\coli\Documents\Data\mapluft_emissionnen") 
file_name = "mapLuft_{year}_v2024.gdb"
maplufts = {year: path / file_name.format(year=year) for year in (2015, 2020, 2022)}
maplufts

In [ ]:
invs = {
    2015: MapLuftZurich(maplufts[2015], remove_josefstrasse_khkw = False,),
    2020: MapLuftZurich(maplufts[2020], remove_josefstrasse_khkw = False,),
    2022: MapLuftZurich(maplufts[2022], remove_josefstrasse_khkw = False,),
    #"2020_without_josefstrasse": MapLuftZurich(maplufts[2020], remove_josefstrasse_khkw = True,),
}

In [ ]:
josephstrasse_2020 = 137478430.0

In [ ]:
def load_zurich_shape(
    zh_raw_file=r"C:\Users\coli\Documents\ZH-CH-emission\Data\Zurich_borders.txt",
    crs_file: int = WGS84,
    crs_out: int = LV95,
) -> Polygon:
    with open(zh_raw_file, "r") as f:
        points_list = eval(f.read())
        zh_poly = Polygon(points_list[0])
        zh_poly_df = gpd.GeoDataFrame(geometry=[zh_poly], crs=crs_file).to_crs(crs_out)
        zh_poly = zh_poly_df.geometry.iloc[0]
        return zh_poly


zh_shape = load_zurich_shape()


In [ ]:
# print the missing cats for each inventories in each year
# This makes sure we don't have discrepenacies between the inventories
all_cats = sum([inv.categories for inv in invs.values()], [])
for year, inv in invs.items():
    print(year)
    print([cat for cat in all_cats if cat not in inv.categories ])

In [ ]:
#for year, inv in invs.items():
#    invs[year] = crop_with_shape(inv, zh_shape, keep_outside=True) 

In [ ]:
ZH_2_GNFR = {
    # PublicPower
    "GNFR_A": [
        "c2201_BHKW_Emissionen",
        "c2301_KHKWKehricht_Emissionen",
        "c2302_KHKWErdgas_Emissionen",
        "c2303_KHKWHeizoel_Emissionen",
    ],
    # Industry
    "GNFR_B": [
        "c3201_Notstromanlagen_Emissionen",
        "c3301_Prozessenergie_Emissionen",
        "c3401_Metallreinigung_Emissionen",
        "c3402_Holzbearbeitung_Emissionen",
        "c3403_Malereien_Emissionen",
        "c3404_Textilreinigung_Emissionen",
        "c3405_Karosserien_Emissionen",
        "c3406_Raeuchereien_Emissionen",
        "c3407_Roestereien_Emissionen",
        "c3408_Druckereien_Emissionen",
        "c3409_Laboratorien_Emissionen",
        "c3410_Bierbrauereien_Emissionen",
        "c3411_Brotproduktion_Emissionen",
        "c3412_MedizinischePraxen_Emissionen",
        "c3413_Gesundheitswesen_Emissionen",
    ],
    # Other stationary combustion (services, residential, agriculture)
    "GNFR_C": [
        "c2101_Oelheizungen_Emissionen",
        "c2102_Gasheizungen_Emissionen",
        "c2103_HolzheizungenLokalisiert_Emissionen",
        "c2104_HolzheizungenDispers_Emissionen",
        "c2105_Warmwassererzeuger_Emissionen",
    ],
    # Fugitives
    "GNFR_D": [
        "c3416_Tankstellen_Emissionen",
    ],
    # Solvents and product use
    "GNFR_E": [
        "c3417_LoesemittelIG_Emissionen",
        "c5101_LoesemittelHH_Emissionen",
    ],
    # Road transport
    "GNFR_F": [
        "c1301_Personenwagen_Emissionen",
        "c1302_Lastwagen_Emissionen",
        "c1303_Motorraeder_Emissionen",
        "c1304_Linienbusse_Emissionen",
        "c1305_Trolleybusse_Emissionen",
        "c1306_StartStopTankatmung_Emissionen",
        "c1307_Lieferwagen_Emissionen",
        "c1308_Reisebusse_Emissionen",
    ],
    # Shipping
    "GNFR_G": [
        "c1101_Linienschiffe_Emissionen",
        "c1102_PrivaterBootsverkehr_Emissionen",
    ],
    # Offroad mobility
    "GNFR_I": [
    #"GNFR_I-Railways": [
        "c1201_BahnPersonenverkehr_Emissionen",
        "c1202_BahnGueterverkehr_Emissionen",
        "c1203_Tramverkehr_Emissionen",
        "c1204_Kleinbahnen_Emissionen",
    #],
    #"GNFR_I-MobileMachinery": [
        # c31xx are construction stuff
        "c3101_MaschinenHochbau_Emissionen",
        "c3102_Bitumen_Emissionen",
        "c3103_FarbenBaustelle_Emissionen",
        "c3104_MaschinenTiefbau_Emissionen",
        "c3105_Strassenbelag_Emissionen",
        "c3419_IndustrielleFZ_Emissionen",
        "c4101_ForstwirtschaftlicheFZ_Emissionen",
        "c4201_LandwirtschaftlicheFZ_Emissionen",
    ],
    # Waste
    "GNFR_J": [
        "c2401_Klaerschlammverwertung_Emissionen",
        "c3418_Vergaerwerk_Emissionen",
        "c3414_Krematorium_Emissionen",
        "c5201_Gruenabfallverbrennung_Emissionen",
        "c5301_HolzoefenKleingarten_Emissionen",
        "c5401_AbfallverbrennungHaus_Emissionen",
    ],
    # AgriLivestock
    "GNFR_K": [
        "c4401_Nutztierhaltung_Emissionen",
    ],
    # AgriOther
    "GNFR_L": [
        "c4301_Nutzflaechen_Emissionen",
    ],
    # Others
    "GNFR_R": [
        "c5501_HausZooZirkustiere_Emissionen",
        "c5601_Feuerwerke_Emissionen",
        "c5701_Tabakwaren_Emissionen",
        "c5801_BrandFeuerschaeden_Emissionen",
        "c6101_Waelder_Emissionen",
        "c6201_Grasflaechen_Emissionen",
        "c6301_Gewaesser_Emissionen",
        "c6401_Blitze_Emissionen",
    ],
}

In [ ]:
# Deep copy the dictionary
ZH_2_GNFR_2015 = copy.deepcopy(ZH_2_GNFR)
ZH_2_GNFR_2015["GNFR_F"].remove("c1307_Lieferwagen_Emissionen")
ZH_2_GNFR_2015["GNFR_F"].remove("c1308_Reisebusse_Emissionen")
ZH_2_GNFR_2015["GNFR_J"].append("c3415_Kompostierung_Emissionen")

cats_groups = {
    #2015: ZH_2_GNFR_2015,
    2015: ZH_2_GNFR,
    2020: ZH_2_GNFR,
    2022: ZH_2_GNFR,
    #"2020_without_josefstrasse": ZH_2_GNFR,
}

groupped_invs = {
    year: group_categories(inv, cats_groups[year]) for year, inv in invs.items()
}

In [ ]:
df_tot = groupped_invs[2022].total_emissions.T.sort_index()
df_tot *= 1e-3 # kg to ton
df_tot['CO2'] *= 1e-3 # ton to kton
df_tot[[
    'CO2',
    'CH4',
    'NOx',
    'CO',
    'BC'
    ]].round(1)


In [ ]:
plt.style.use('default')

substances = ['CO2', 'CH4', 'NOx', 'BC', 'CO']
fig, axes = plt.subplots(nrows=len(substances), figsize=(8,12), gridspec_kw={'hspace': 0.0}, sharex=True)
categories = [
        #"c1301_Personenwagen_Emissionen_Kanton",
        #"c1302_Lastwagen_Emissionen_Kanton",
        #"c1303_Motorraeder_Emissionen_Kanton",
        #"c1304_Linienbusse_Emissionen_Kanton",
        #"c1305_Trolleybusse_Emissionen_Kanton",
        #"c1306_StartStopTankatmung_Emissionen_Kanton",
        #"c1307_Lieferwagen_Emissionen_Kanton",
        #"c1308_Reisebusse_Emissionen_Kanton",
]
categories = [
    "GNFR_A",
    "GNFR_B",
    "GNFR_C",
    "GNFR_F",
    #"GNFR_G",
    "GNFR_I",
    "GNFR_J",
    "GNFR_K",
    #"GNFR_L",
    "GNFR_R",
    "__total__",
]
offsets = {
    2015: -0.2,
    2020: 0,
    2022: 0.2,

}
# Use some fancy colors
colors = {
    2015: '#1b9e77',
    2020: '#d95f02',
    2022: '#7570b3',
}

for i, substance in enumerate(substances):
    ax = axes[i]
    for year in [2015, 2020, 2022]:
        df_tot = groupped_invs[year].total_emissions.T.sort_index()
        df_tot *= 1e-3 # kg to ton
        df_tot['CO2'] *= 1e-3 # ton to kton
        if substance not in df_tot:
            continue
        # Add missing categories to the df as 0
        for cat in categories:
            if cat not in df_tot.index:
                df_tot.loc[cat] = 0
        # Plot a bar for each year
        x_axis = np.arange(len(categories)) + offsets[year]
        ax.bar(
            x_axis,
            df_tot[substance][categories],
            width=0.2,
            label=year,
            color=colors[year],
        )

    # No ticks 
    ax.set_xticks([])

    # Units 
    unit = 'kt' if substance == 'CO2' else 't'
    ax.set_ylabel(f"{substance} [{unit}]")


    
# just align the last column of axes:
fig.align_ylabels(axes)

ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(categories)
# Rotate 
ax.set_xticklabels(categories, rotation=90)
ax.legend()
plt.show()

In [ ]:
total_emissions_gnrf = {year: get_total_emissions(inv)['CO2'] for year, inv in groupped_invs.items()}
total_emissions = {year: get_total_emissions(inv)['CO2'] for year, inv in invs.items()}

In [ ]:
total_emissions_2015 = total_emissions[2015]
total_emissions_2020 = total_emissions[2020]
for cat in all_cats:
    if cat not in total_emissions_2015 or cat not in total_emissions_2020:
        continue
    # get the relative difference between 2015 and 2020
    diff = (total_emissions_2020[cat] - total_emissions_2015[cat]) / total_emissions_2020[cat]
    print(f"{cat}: {diff * 100:.2f}%")
        


### Extrapolate emissions to the future


Warning: This was done to estimate 2022 when we did not have the 2022 inventory.

Now we have it so we can use it directly.
This is still there in case someone wants to use it for other years.

In [ ]:
total_emissions
year_future = 2022
df = pd.DataFrame(total_emissions).T / 1000
# Correct the value of 2020 because of COVID, 4% less is in the inventory
covid_corrections = {
    # "civil_categories
    'c1301_Personenwagen_Emissionen_Kanton': 1.0462,
    'c1303_Motorraeder_Emissionen_Kanton': 1.0462,
    'c1308_Reisebusse_Emissionen_Kanton': 1.0462, 
    'c1306_StartStopTankatmung_Emissionen_Kanton': 1.0462,
    # Heavy trafic
    "c1302_Lastwagen_Emissionen_Kanton": 1.0154,
    # Taken comparing km runned by public boats in 2015 and 2020
    "c1101_Linienschiffe_Emissionen_Kanton": 1.391,
}

for cat, correction in covid_corrections.items():
    df.loc[2020, cat] *= correction



# Linearly extrapolate to get 2022
df.loc[year_future] = df.loc[2020] + (df.loc[2020] - df.loc[2015]) / 5 * (year_future - 2020)
# If nan assumes same as 2020
mask_nan = df.loc[year_future].isna()
df.loc[year_future, mask_nan] = df.loc[2020, mask_nan]

# Clip to 0 
df[df < 0] = 0

# Calculate the scaling factors for how to get inventory of 2022 based on 2020 


scaling_factors = df.loc[2022] / df.loc[2020]

# Apply the correct assuming 2022 was not a covid year
for cat, correction in covid_corrections.items():
    scaling_factors[cat] *= correction
    # remove the 2020 correction
    df.loc["2020_not_corrected", cat] = df.loc[2020, cat] / correction



df.plot(figsize=(10, 5), title='Total emissions in kt/year')

In [ ]:
df.index.name = 'year'
df[covid_corrections.keys()].T

In [ ]:
{'CO2': dict(scaling_factors)}

In [ ]:
# Scale the 2020 inventories with the calculated scaling factors to have the 2022
inv_2022 = scale_inventory(invs[2020], scaling_dict={'CO2': scaling_factors})
inv_2022_GNRF = group_categories(inv_2022, ZH_2_GNFR)

total_emissions_gnrf[2022] = get_total_emissions(inv_2022_GNRF)['CO2']
total_emissions_gnrf[2022]['GNFR_A'] -= josephstrasse_2020

In [ ]:
# Show the differences in total emissions from the different yeas
df = pd.DataFrame(total_emissions_gnrf) / 1000
# Iterate ovr the rows of the col and format the numbers as normal floats
year = 2022
for line in df[year].iteritems():
    print(line[0], '{:.0f}'.format(line[1]), '{:.1f}%'.format(line[1] / df.loc['__total__',year] * 100), )

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.style.use("seaborn-whitegrid")

tt = total_emissions_gnrf.copy()
#tt.pop('2020_without_josefstrasse')
df = pd.DataFrame(tt).T / 1000
# Drop the __total__ column
df = df.drop(columns=['__total__'])
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
df.T.plot.bar(ax=ax)
ax.set_ylabel("Emissions [t/y]")

### Saving to a raster

Following cells are saving the raster to a file. 

In [ ]:
# Save to a raster 

# edge of the raster cells
RASTER_EDGE = 100


def load_zurich_shape(
    zh_raw_file=r"C:\Users\coli\Documents\ZH-CH-emission\Data\Zurich_borders.txt",
    crs_file: int = WGS84,
    crs_out: int = LV95,
) -> Polygon:
    with open(zh_raw_file, "r") as f:
        points_list = eval(f.read())
        zh_poly = Polygon(points_list[0])
        zh_poly_df = gpd.GeoDataFrame(geometry=[zh_poly], crs=crs_file).to_crs(crs_out)
        zh_poly = zh_poly_df.geometry.iloc[0]
        return zh_poly


zh_shape = load_zurich_shape()

x_min, y_min, x_max, y_max = zh_shape.bounds
d = RASTER_EDGE
xmin, ymin = floor(x_min) // d * d, floor(y_min) // d * d
nx, ny = int(x_max - xmin) // d, int(y_max - ymin) // d
xs = np.arange(xmin, xmin + nx * d, step=d)
ys = np.arange(ymin, ymin + ny * d, step=d)
point_x = np.repeat(xs, ny)
point_y = np.tile(ys, nx)

polys = [
    Polygon(((x, y), (x + d, y), (x + d, y + d), (x, y + d)))
    for y in reversed(ys)
    for x in xs
]
centers = [Point((x + d / 2, y + d / 2)) for y in reversed(ys) for x in xs]
zh_gdf = gpd.GeoDataFrame(geometry=polys, crs=LV95)
gdf_centers = gpd.GeoDataFrame(geometry=centers, crs=LV95)

# %% prepare the output on WGS84
WGS84_gdf = zh_gdf.to_crs(WGS84)
for i in range(4):
    WGS84_gdf[f"coord_{i}"] = WGS84_gdf.geometry.apply(
        lambda poly: poly.exterior.coords[i]
    )







In [ ]:
rasters_inv = remap_inventory(
    #crop_with_shape(inv_2022_GNRF, zh_shape),
    inv_2022_GNRF,
    zh_gdf.geometry,
)


In [ ]:

from emiproc.exports.netcdf import nc_cf_attributes

ds_out = xr.Dataset(
    coords={
        "x": (
            "x",
            xs + d / 2,
            {
                "standard_name": "easting",
                "long_name": "easting",
                "units": "m",
                "comment": "center_of_cell",
                "projection": "Swiss coordinate system LV95",
            },
        ),
        "y": (
            "y",
            (ys + d / 2)[::-1],
            {
                "standard_name": "northing",
                "long_name": "northing",
                "units": "m",
                "comment": "center_of_cell",
                "projection": "Swiss coordinate system LV95",
            },
        ),
        "lon": (
            ("y", "x"),
            gdf_centers.to_crs(WGS84)
            .geometry.apply(lambda point: point.coords[0][0])
            .to_numpy()
            .reshape((ny, nx)),
            {
                "standard_name": "longitude",
                "long_name": "longitude",
                "units": "degrees_east",
                "comment": "center_of_cell",
                "bounds": "lon_bnds",
                "projection": "WGS84",
            },
        ),
        "lat": (
            ("y", "x"),
            gdf_centers.to_crs(WGS84)
            .geometry.apply(lambda point: point.coords[0][1])
            .to_numpy()
            .reshape((ny, nx)),
            {
                "standard_name": "latitude",
                "long_name": "latitude",
                "units": "degrees_north",
                "comment": "center_of_cell",
                "bounds": "lat_bnds",
                "projection": "WGS84",
            },
        ),
        "lon_bnds": (
            ("nv", "y", "x"),
            np.array(
                [
                    WGS84_gdf[f"coord_{i}"]
                    .apply(lambda p: p[0])
                    .to_numpy()
                    .reshape((ny, nx))
                    for i in range(4)
                ]
            ),
            {
                "comment": "cell boundaries, anticlockwise",
            },
        ),
        "lat_bnds": (
            ("nv", "y", "x"),
            np.array(
                [
                    WGS84_gdf[f"coord_{i}"]
                    .apply(lambda p: p[1])
                    .to_numpy()
                    .reshape((ny, nx))
                    for i in range(4)
                ]
            ),
            {
                "comment": "cell boundaries, anticlockwise",
            },
        ),
        "source_category": ("source_category", rasters_inv.categories, {}),
    },
    data_vars={
        f"emi_{sub}": (
            ("source_category", "y", "x"),
            np.full((len(rasters_inv.categories), ny, nx), np.nan),
            {
                "standard_name": f"tendency_of_atmosphere_mass_content_of_{sub}_due_to_emission",
                "long_name": f"Emissions of {sub}",
                "units": "μg m-2 s-1",
                "comment": "annual mean emission rate",
            },
        )
        for sub in rasters_inv.substances
    }
    | {
        f"emi_{sub}_total": (
            ("source_category"),
            np.full(len(rasters_inv.categories), np.nan),
            {
                "long_name": f"Total Emissions of {sub}",
                "units": "kg yr-1",
            },
        )
        for sub in rasters_inv.substances
    }
    | {
        f"emi_{sub}_all_sectors": (
            ("y", "x"),
            np.zeros((ny, nx)),
            {
                "standard_name": f"tendency_of_atmosphere_mass_content_of_{sub}_due_to_emission",
                "long_name": f"Aggregated Emissions of {sub} from all sectors",
                "units": "μg m-2 s-1",
                "comment": "annual mean emission rate",
            },
        )
        for sub in rasters_inv.substances
    },
    attrs=nc_cf_attributes(
        author="Lionel Constantin, Empa",
        contact="dominik.brunner@empa.ch",
        title="Extrapolated annual mean emissions of CO2 of the city of Zurich (all emissions reported in the mapluft inventory, included in the raster boundaries)",
        source="https://www.stadt-zuerich.ch/gud/de/index/umwelt_energie/luftqualitaet/schadstoffquellen/emissionskataster.html",
        comment="Created for use in the EU project ICOS-Cities",
        history="Created from original GIS inventory mapLuft of the city of Zurich by rasterizing all point, line and area sources",
        additional_attributes={
            "swiss_coordinate_system_lv95": "https://www.swisstopo.admin.ch/en/knowledge-facts/surveying-geodesy/coordinates/swiss-coordinates.html",
            "comment_lv95": "In original LV95 system, x denote northings and y eastings. They have been exchanged here for better compatibility with lon/lat.",
            "copyright_notice": "",
            "emiproc_history": str(rasters_inv.history),
            "extrapolation_method": (
                "We compared the inventory of 2015 and 2020 and assumed each category was linearly continuing its trend. "
                "One incinerator was removed as is ceased to operate in 2021. "
                "A correction was applied to the traffic emissions to account for the decrease in traffic due to the COVID-19 pandemic. "
            )
        },
    ),
)


In [ ]:
for category, sub in rasters_inv._gdf_columns:
    if (category, sub) not in rasters_inv._gdf_columns:
        continue
    emissions = rasters_inv.gdf[(category, sub)].to_numpy().reshape((ny, nx))
    # convert from kg/y to μg m-2 s-1
    rescaled = emissions * 1e9 / SEC_PER_YR / (RASTER_EDGE * RASTER_EDGE)
    # Assign emissions of this category
    ds_out[f"emi_{sub}"].loc[dict(source_category=category)] = rescaled
    # Add the the categories aggregated emission
    ds_out[f"emi_{sub}_all_sectors"] += rescaled
    # Add to the total emission value
    ds_out[f"emi_{sub}_total"].loc[dict(source_category=category)] = np.sum(emissions)


In [ ]:
ds_out

In [ ]:
outdir = Path(r"C:\Users\coli\Documents\emiproc\files\outputs")
ds_out.to_netcdf(
    outdir
    / f"zurich_{RASTER_EDGE}x{RASTER_EDGE}_{year_future}_scaled_from_{maplufts[2020].stem[:-6]}_v0.1.nc"
)